# Most Vulnerable Projects - Data analysis

This notebook aims to find the most vulnerable projects based on control flow graph (CFG) metrics extracted from code changes. It processes a dataset of code changes, dot-formatted CFGs, and computes various metrics to identify projects with potentially risky code changes.

This Notebook answers these questions:

1. From the selected projects, what are the top 10 that have the most commits on fixing bugs, issues, and vulnerabilities?

    1. What are their common characteristics? For example, are they all related to machine learning, to mobile development, etc?

    1. Is there anything that all the top projects don't have in common?

    1. Do any of those projects have incivil commits? By incivil comments, we mean commit messages containing offensive, rude, or hostile language (e.g., as detected by a toxicity classifier or manual review).

    1. Regarding the CFGs of those projects, is the depth of the CFG correlated in any way to the existence of bugs, vulnerabilities, and issues?


In [ ]:
# preamble
import pandas as pd
import os
import re
from IPython.display import display, SVG
import warnings
warnings.filterwarnings('ignore', category=pd.errors.ParserWarning)

In [ ]:
%load_ext autoreload
%autoreload 2

import sys
import os
# Ensure the `utils` module is on the Python path
sys.path.insert(0, os.path.abspath('utils'))

In [ ]:
data_dir = "../assets/data-samples/"
csv_files = [f for f in os.listdir(data_dir) if f.endswith('.csv')]
print(f"Found {len(csv_files)} CSV files:")
for f in sorted(csv_files):
    size_mb = os.path.getsize(os.path.join(data_dir, f)) / (1024**2)
    print(f"  - {f} ({size_mb:.1f} MB)")

In [ ]:
# Load the CSV files into a single DataFrame
if len(csv_files) == 0:
    raise FileNotFoundError(f"No CSV files found in directory: {data_dir}")
sample_file = os.path.join(data_dir, csv_files[-1])
# Read the CSV with low_memory=False to avoid DtypeWarning from mixed types across chunks
# and explicitly set common text columns to string dtype to ensure consistency
df = pd.read_csv(sample_file, dtype={
  'project_name': 'string',
  'project_description': 'string',
  'project_url': 'string',
  'project_creation_date': 'string',
  'project_database': 'string',
  'project_languages': 'string',
  'project_oss': 'string',
  'project_topics': 'string',
  'commit_url': 'string',
  'file_path': 'string',
  'method_name': 'string',
  'cfg_dot': 'string',
  'cfg_state': 'string'
})

initial_len = len(df)
print(f"Loaded {initial_len} CFG entries from {os.path.basename(sample_file)}")
print(df.info())
print(df.head())

In [ ]:
from utils.is_column_empty import is_effectively_empty

cols_to_drop = [col for col in df.columns if is_effectively_empty(df[col])]

if cols_to_drop:
    print(f"Removing effectively empty columns: {cols_to_drop}")
    df = df.drop(columns=cols_to_drop)
    
print(f"DataFrame now has {df.shape[1]} columns after removing empty ones.")
# print the datatyps of the remaining columns
print(df.dtypes)

In [ ]:
# --- make the project language column more usable ---
from utils.parse_language_series import split_semicolon_series
df['project_languages'] = split_semicolon_series(df['project_languages'])

print(f"Unique programming languages found: {df['project_languages'].explode().nunique()}")
print(df.info())

## Top 10 projects with most commits fixing bugs, issues, and vulnerabilities

**Question:** From the selected projects, what are the top 10 that have the most commits on fixing bugs, issues, and vulnerabilities?

To answer this question, we will take this approach:

- Remove the rows with `PRE` in the `cfg_state` column, as they do not represent actual code changes.
- Group the DataFrame by `project_name` and count the number of commits for each project.
- Sort the projects by the number of commits in descending order and select the top 10.

In [ ]:
from IPython.display import display

# Remove "PRE" rows (non-change states) and count unique commits per project
df_changes = df[~df['cfg_state'].str.upper().eq('PRE')].copy()
removed_count = len(df) - len(df_changes)
print(f"Removed {removed_count} PRE rows -> {len(df_changes)} rows remaining (of {len(df)})")

# Unique commit count per project (a commit may touch multiple files/methods; count unique commit_url)
commit_counts = df_changes.groupby('project_name')['commit_url'].nunique().reset_index(name='num_commits')

# Additional useful aggregations for context:
agg = df_changes.groupby('project_name').agg(
  project_description=('project_description', lambda s: s.dropna().iloc[0] if s.notna().any() else pd.NA),
  project_languages=('project_languages', 'first'),
  project_url=('project_url', 'first'),
  total_files_changed=('files_changed_count', 'sum'),
  unique_files_changed=('file_path', 'nunique'),
  unique_methods_changed=('method_name', 'nunique')
).reset_index()

# Merge commit counts with aggregated metadata and sort
project_stats = commit_counts.merge(agg, on='project_name')
project_stats = project_stats.sort_values('num_commits', ascending=False)

# Top 10 projects by number of unique commits fixing issues/bugs/vulns
top_n = 10
top_projects = project_stats.head(top_n).reset_index(drop=True)

# Add percent of total unique commits to give context
total_unique_commits = commit_counts['num_commits'].sum()
top_projects['pct_of_total_commits'] = (top_projects['num_commits'] / total_unique_commits * 100).round(2)

display(top_projects)


**Potential Improvements:**
The last column `pct_of_total_commits` could be combined with full context. For example the above figure we can see that [ceylon/ceylon-compiler](https://github.com/ceylon/ceylon-compiler ) accounts for **36%** of the total commits fixing bugs, issues, and vulnerabilities. However, without knowing the total number of commits in a project, this percentage alone does not provide a complete picture of the project's overall activity or the significance of bug-fixing commits relative to its total development efforts.

## Characteristics of Top 10 Projects

**Question:** What are their common characteristics? For example, are they all related to machine learning, to mobile development, etc?

**Question:** Is there anything that all the top projects don't have in common?


As we can see from the table, all the top 10 projects are: 

#### Top 10 Projects

| Rank | Project Name | Num Commits | Description | Primary Languages |
|------|--------------|-------------|-------------|-------------------|
| 1 | `ceylon/ceylon-compiler` | 2126 | Ceylon compiler (Java backend) | Perl, Ruby, Shell, C, Java... |
| 2 | `aokpx-private/platform_packages_apps_Calendar` | 1040 | (Android Calendar App) | Java |
| 3 | `dana-i2cat/opennaas-routing-nfv` | 679 | OpenNaaS Routing NFV | Java, CSS, JS, Shell |
| 4 | `rfkrocktk/red5-server` | 630 | Live Video Streaming Server | Java, Shell, JS |
| 5 | `eclipse/webtools.jsf` | 595 | Eclipse Web Tools Platform | Java, CSS |
| 6 | `ebayopensource/turmeric-runtime` | 66 | Turmeric SOA Runtime | Java, Shell, Perl |
| 7 | `mibto/mez` | 58 | Zeiterfassung Metzler (Time Tracking) | Java |
| 8 | `Ourobor/petulant-batman` | 49 | Group Project for an SE Class | Java |
| 9 | `venukumar/bartsy-venue-android` | 47 | (Android App) | Java |
| 10 | `ovitas/compass2` | 30 | Compass 2 for Sesam 4 | Java, JS |

---

### Similarities

1.  **Dominance of Java**:
    *   **All 10 projects** utilize **Java** as a primary or significant language. This indicates a strong homogeneity in the underlying technology stack of the most "fix-heavy" projects in this dataset. It suggests the control flow graph analysis or the dataset collection process might have been heavily focused on Java repositories.
    
1.  **Infrastructure & Tooling Orientation**:
    -   A significant portion of the top projects are **developer tools, frameworks, or infrastructure** rather than simple end-user applications.
        -   *Compiler*: `ceylon-compiler`
        -   *Server*: `red5-server`
        -   *Dev Tools*: `webtools.jsf`
        -   *Framework*: `turmeric-runtime`, `opennaas-routing-nfv`   

This suggests that complex infrastructure projects might require more frequent bug fixing or have more rigorous commit practices.

---


### Dissimilarities

1.  **Scale of Activity (The "Power Law" Distribution)**:
    -   There is a **massive disparity** in fix volume between the top and the bottom of this list.
    -   The top project (`ceylon-compiler`) has **2,126** fix commits.
    -   The 6th project (`turmeric-runtime`) drops sharply to **66** commits.
    -   The 10th project (`compass2`) has only **30** commits.
    -   This indicates that the dataset is dominated by a handful of extremely active projects, while the "long tail" consists of much smaller or less active repositories.

1.  **Project Governance & Maturity**:
    -   **Corporate/Foundation**: Projects like `eclipse/webtools.jsf` (Eclipse Foundation) and `ebayopensource/turmeric-runtime` (eBay) likely follow strict corporate or foundation governance models.
    -   **Community/Indie**: Projects like `rfkrocktk/red5-server` (Red5) are community-driven open source.
    -   **Student/Academic**: `Ourobor/petulant-batman` is explicitly described as a "Group Project for an SE Class", representing a vastly different level of maturity and code quality standards compared to Eclipse or eBay.

1.  **Application Domain**:
    -   The functional domains are quite distinct:
        -   **Language Engineering**: `ceylon-compiler`
        -   **Mobile/Android**: `platform_packages_apps_Calendar`, `bartsy-venue-android`
        -   **Media Streaming**: `red5-server`
        -   **Networking/NFV**: `opennaas-routing-nfv`
        -   **Enterprise SOA**: `turmeric-runtime`

This diversity shows that high bug-fix counts are not unique to one specific industry vertical.


While the top 10 projects share a common technical foundation, they represent a highly diverse set of governance models, domains, and scales. The presence of both enterprise-grade foundations (Eclipse) and student projects (petulant-batman) in the top 10 highlights the varied nature of the dataset. The extreme skew in commit counts (2000+ vs <70) suggests that any aggregate analysis should be careful not to be overwhelmed by the top 1-5 dominant projects.

## Incivil Commits in Top Projects

**Question:** Do any of those projects have incivil commits? By incivil comments, we mean commit messages containing offensive, rude, or hostile language (e.g., as detected by a toxicity classifier or manual review).

To determine if any of the top projects have incivil commits, we can analyze the commit messages associated with each commit in the dataset. I used a pre-trained toxicity classifier (e.g., from Hugging Face Transformers `unitary/toxic-bert` model) to identify potentially incivil language in commit messages.

First, we load the toxicity classifier and define a function to classify commit messages

The settings that I used for the toxicity classifier are as follows:

- **Model:** `unitary/toxic-bert`
- **Threshold:** 0.7 (A commit message is considered incivil if the toxicity score exceeds this threshold)
- **Device:** MPS (Apple Silicon) if available, otherwise CPU

In [ ]:
from transformers import pipeline
import torch

#  Pre-check MPS — safer and more explicit
device = 0 if torch.backends.mps.is_available() else -1
print(f"Using device: {'MPS' if device == 0 else 'CPU'}")

classifier = pipeline(
    "text-classification",
    model="unitary/toxic-bert",
    device=device,
    top_k=None,
    truncation=True,          # ← critical: avoids errors on long commit messages
    batch_size=8,             # ← enables batching for faster apply()
)

def is_toxic(text, threshold=0.7):
    try:
        if not isinstance(text, str) or not text.strip():
            return False
        # classifier returns list of list of dicts: [[{label, score}, ...], ...]
        preds = classifier(text)[0]  # list of label-score dicts for one input
        toxic_score = next((r['score'] for r in preds if r['label'] == 'toxic'), 0.0)
        return toxic_score > threshold
    except Exception as e:
        # Optional: log problematic entries (e.g., encoding issues)
        # print(f"Skipped text (len={len(text)}): {e}")
        return False  # conservative: treat failures as non-toxic
    
top_project_names = set(top_projects['project_name'])
df_top_projects = df[df['project_name'].isin(top_project_names)].copy()
df_top10_unique_commits = df_top_projects.drop_duplicates(subset=['commit_url']).copy()

print(f"Filtered to {len(df_top_projects)} rows ({len(df_top10_unique_commits)} unique commits) from top {len(top_project_names)} projects")

# Now apply toxicity classifier just once per unique commit (more efficient + avoids redundancy)
from tqdm.auto import tqdm
tqdm.pandas()
df_top10_unique_commits['is_toxic'] = df_top10_unique_commits['commit_message'].progress_apply(is_toxic)

# print top five toxic commits for inspection
display(df_top10_unique_commits[df_top10_unique_commits['is_toxic']].head(5)[['project_name', 'commit_url', 'commit_message']])

toxic_counts = df_top10_unique_commits.groupby('project_name')['is_toxic'].agg(
    toxic_commit_count='sum',
    total_commits='size'  # same as nunique(commit_url) here, since deduped
).reset_index()

top_projects_enhanced = top_projects.merge(toxic_counts, on='project_name')
top_projects_enhanced['toxic_ratio'] = (
    top_projects_enhanced['toxic_commit_count'] / top_projects_enhanced['num_commits']
).round(4)


display(top_projects_enhanced[['project_name', 'num_commits', 'toxic_commit_count', 'toxic_ratio']])

**Decision:**
There are no incivil commits in the top 10 projects based on the toxicity analysis of commit messages using the `unitary/toxic-bert` model.

**Potential Improvements:**
- Consider using multiple toxicity detection models to cross-validate results and improve accuracy.
- Manually review a sample of commit messages flagged as incivil to ensure the model's accuracy and relevance to the context of software development. 

## Correlation Between CFG Depth and Bugs/Vulnerabilities

**Question:** Regarding the CFGs of those projects, is the depth of the CFG correlated in any way to the existence of bugs, vulnerabilities, and issues?

To analyze the correlation between CFG depth and the existence of bugs, vulnerabilities, and issues, we can follow these steps:

- We’ll treat a method change unit as a unique (`project_name`, `commit_url`, `file_path`, `method_name`) that has exactly one `pre` and one `post` entry.
- Compute Depth (and other metrics) for Both CFGs
- Associate the matrices with keywords like bug, vulnerability, issue from commit messages
- Now you can ask nuanced questions:

    - Do security fixes reduce depth more than generic bug fixes?
    - Do refactors reduce depth the most?
    - Do features increase depth?

In [ ]:
# print(df.info())

df_clean = df[
    df['cfg_state'].isin(['PRE', 'POST']) &
    df['cfg_dot'].notna()
].copy()

df_clean['method_key'] = (
    df_clean['project_name'] + '|' +
    df_clean['commit_url'] + '|' +
    df_clean['file_path'] + '|' +
    df_clean['method_name']
)

state_counts = df_clean.groupby('method_key')['cfg_state'].value_counts().unstack(fill_value=0)


paired_keys = state_counts[(state_counts['PRE'] == 1) & (state_counts['POST'] == 1)].index
paired_df = df_clean[df_clean['method_key'].isin(paired_keys)].copy()

# Statistics on paired methods
num_paired_methods = len(paired_keys)
num_unique_projects = paired_df['project_name'].nunique()
print(f"Found {num_paired_methods} paired methods across {num_unique_projects} unique projects.")

print(paired_df.info())
print(f"Found {len(paired_df)} total rows in paired DataFrame.")

In [ ]:
## Now we pivot the table

wide = paired_df.pivot(
    index='method_key',
    columns='cfg_state',
    values=['cfg_dot', 'project_name', 'commit_url', 'file_path', 'method_name', 'commit_message']
).reset_index()

# Flatten MultiIndex columns
wide.columns = [
    f"{col[0]}_{col[1]}" if col[1] else col[0]
    for col in wide.columns
]

# Final tidy dataset
changes = wide[[
    'project_name_PRE', 'commit_url_PRE', 'file_path_PRE', 'method_name_PRE',
    'commit_message_PRE', 'cfg_dot_PRE', 'cfg_dot_POST'
]].rename(columns={
    'project_name_PRE': 'project_name',
    'commit_url_PRE': 'commit_url',
    'file_path_PRE': 'file_path',
    'method_name_PRE': 'method_name',
    'commit_message_PRE': 'commit_message',
    'cfg_dot_PRE': 'cfg_pre',
    'cfg_dot_POST': 'cfg_post'
}).reset_index(drop=True)

print(f"Final changes DataFrame has {len(changes)} rows.")
print(changes.info())
display(changes.head())

In [ ]:
from utils.cfg_utils import dot_to_graph, get_depth, count_nodes_edges

from tqdm import tqdm
tqdm.pandas()

changes['G_pre'] = changes['cfg_pre'].progress_apply(dot_to_graph)
changes['G_post'] = changes['cfg_post'].progress_apply(dot_to_graph)


changes['nodes_pre'], changes['edges_pre'] = zip(*changes['G_pre'].apply(count_nodes_edges))
changes['depth_pre'] = changes['G_pre'].apply(get_depth)

changes['nodes_post'], changes['edges_post'] = zip(*changes['G_post'].apply(count_nodes_edges))
changes['depth_post'] = changes['G_post'].apply(get_depth)

changes['delta_depth'] = changes['depth_post'] - changes['depth_pre']

In [ ]:
# Ensure commit_message is string (handle NaNs)
changes['commit_message'] = changes['commit_message'].fillna("").astype(str)

from utils.classifier import classify_fix_type

# Apply
changes['fix_type'] = changes['commit_message'].apply(classify_fix_type)
changes['is_fix'] = changes['fix_type'] != "other"

In [ ]:
# Quick sanity check
print("Fix type distribution:")
print(changes['fix_type'].value_counts().sort_index())

# Example: top 5 security-fixing commits with largest depth reduction
(
    changes[changes['fix_type'] == 'security']
    .sort_values('delta_depth')
    .head(5)[['project_name', 'file_path', 'method_name', 'commit_message', 'depth_pre', 'depth_post', 'delta_depth']]
)

In [ ]:
print("Depth = 0:", (changes['depth_pre'] == 0).sum(), "/", len(changes))
print("Depth = 1:", (changes['depth_pre'] == 1).sum())
print("Depth ≥ 5:", (changes['depth_pre'] >= 5).sum())

In [ ]:
# Define buckets
def depth_bucket(d):
    if d <= 2:
        return 'shallow (≤2)'
    elif d <= 5:
        return 'medium (3–5)'
    elif d <= 10:
        return 'moderate (6–10)'
    elif d <= 20:
        return 'deep (11–20)'
    else:
        return 'very_deep (>20)'

changes['depth_bucket'] = changes['depth_pre'].apply(depth_bucket)

# Analyze by bucket + fix_type
bucket_analysis = (
    changes
    .groupby(['depth_bucket', 'fix_type'])
    .agg(
        count=('delta_depth', 'size'),
        delta_depth_mean=('delta_depth', 'mean'),
        delta_depth_median=('delta_depth', 'median'),
        pct_decreased=('delta_depth', lambda x: (x < 0).mean() * 100)
    )
    .round(3)
    .reset_index()
)

# Pivot for readability
pivot = bucket_analysis.pivot(
    index='depth_bucket',
    columns='fix_type',
    values='delta_depth_mean'
).round(2)

print(pivot)

In [ ]:
import numpy as np
import pandas as pd
from scipy import stats

# Helper: robust mode (returns scalar)
def mode_func(x):
    try:
        m = stats.mode(x.dropna(), keepdims=False)
        return int(m.mode) if m.count > 0 else np.nan
    except:
        return np.nan

# Define aggregation specs for ONE column (reusable)
def get_agg_spec(col_name: str):
    return {
        f'{col_name}_count': (col_name, 'size'),
        f'{col_name}_mean': (col_name, 'mean'),
        f'{col_name}_median': (col_name, 'median'),
        f'{col_name}_mode': (col_name, mode_func),
        f'{col_name}_std': (col_name, 'std'),
        f'{col_name}_min': (col_name, 'min'),
        f'{col_name}_max': (col_name, 'max'),
        f'{col_name}_q25': (col_name, lambda x: np.percentile(x, 25)),
        f'{col_name}_q75': (col_name, lambda x: np.percentile(x, 75)),
        f'{col_name}_skew': (col_name, lambda x: stats.skew(x, nan_policy='omit')),
        f'{col_name}_kurtosis': (col_name, lambda x: stats.kurtosis(x, nan_policy='omit')),
        f'{col_name}_pct_ge_5': (col_name, lambda x: (x >= 5).mean() * 100),
        f'{col_name}_pct_ge_10': (col_name, lambda x: (x >= 10).mean() * 100),
        f'{col_name}_pct_ge_15': (col_name, lambda x: (x >= 15).mean() * 100),
        f'{col_name}_pct_ge_20': (col_name, lambda x: (x >= 20).mean() * 100),
    }

# Apply separately to depth_pre and depth_post
pre_agg = changes.groupby('fix_type').agg(**get_agg_spec('depth_pre'))
post_agg = changes.groupby('fix_type').agg(**get_agg_spec('depth_post'))

# Merge
summary = pd.concat([pre_agg, post_agg], axis=1)

# Now add delta stats (safe way)
delta_stats = changes.groupby('fix_type').agg(
    delta_mean=('delta_depth', 'mean'),
    delta_median=('delta_depth', 'median'),
    delta_std=('delta_depth', 'std'),
    pct_reduced=('delta_depth', lambda x: (x < 0).mean() * 100),
    pct_increased=('delta_depth', lambda x: (x > 0).mean() * 100),
    pct_unchanged=('delta_depth', lambda x: (x == 0).mean() * 100),
)

summary = summary.join(delta_stats).round(2).reset_index()

# Optional: reorder columns for readability
priority_cols = [
    'fix_type',
    'depth_pre_mean', 'depth_post_mean',
    'depth_pre_median', 'depth_post_median',
    'depth_pre_mode', 'depth_post_mode',
    'depth_pre_max', 'depth_post_max',
    'depth_pre_pct_ge_10', 'depth_post_pct_ge_10',
    'depth_pre_pct_ge_20', 'depth_post_pct_ge_20',
    'delta_mean', 'delta_median', 'pct_reduced', 'pct_increased'
]

# Keep only columns that exist (in case some agg failed)
existing_cols = [c for c in priority_cols if c in summary.columns]
summary = summary[existing_cols]

print(summary.to_string(index=False, float_format="%.2f"))

| Metric | Key Finding |
|--------|-------------|
| Pre-fix depth | Security and crash-related fixes occur in significantly deeper methods: <br> - security: median = 5, mean = 8.62 <br> - crash: median = 6, mean = 10.74 <br> - bug: median = 3, mean = 6.29 <br> - Vulnerabilities and crashes concentrate in more complex control flows. |
| Depth change (Δ) | Mean Δdepth ≈ 0 for all categories, but direction matters: <br> - security: -0.01 (only category with negative mean) <br> - pct_reduced > pct_increased for security (1.00% vs 1.40%) — slight bias toward simplification <br> - crash: Δ = 0.00, but pct_reduced = 1.96% < pct_increased = 3.67% → net increase |
| Tail complexity (≥20) | Security fixes: 9.01% → 8.78% (−0.23 pp) <br> Crash fixes: 14.11% → 14.19% (+0.08 pp) <br> → Only security fixes consistently reduce extreme complexity. |
| Max depth unchanged | depth_pre_max == depth_post_max for all (e.g., security: 123→123) <br> - Top outliers are not being simplified — possibly due to architectural constraints or incomplete refactorings. |

## Interpretation
- Confirmation of complexity-risk hypothesis:
Methods involved in security and crash fixes are 30–80% deeper (by median) than generic bug fixes — supporting the idea that control flow complexity contributes to high-severity defects.
- Subtle, not structural, simplification:
    - The near-zero Δdepth suggests most fixes are localized patches, not full refactors:
    - Adding a null-check (if (x != null)) → +1 depth
    - Removing a redundant branch → −1 depth
    - Net effect cancels out on average.
- Security fixes are unique:
They are the only category showing:
    - Negative mean Δdepth (−0.01)
    - Reduction in % of methods ≥20 depth (9.01% → 8.78%)
    - Suggests intentional simplification is part of secure coding practice (e.g., reducing branch conditions to eliminate att